ISING MODEL


In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

In [2]:
def lattice_f(N):
    # flat lattice of ±0.5 spins
    lattice = [0.5 * (2*random.randint(0,1)-1) for _ in range(N**2)]
    return lattice

def energy_f(lattice, N, eps=1.0, B=0.0):
    # For ±1/2 spin, scale eps and B so that eps and B refer to the original ±1 model (needed to compare with theory later on)
    # eps_eff = 4 * eps (since s_i*s_j scales by 1/4) and B_eff = 2 * B (since s scales by 1/2).
    eps_eff = 4.0 * eps
    B_eff = 2.0 * B
    pair_sum = 0.0
    field_sum = 0.0
    nsites = N**2
    for i in range(nsites):
        s = lattice[i]
        neighbour = (lattice[(i + N) % nsites] + lattice[(i + 1) % nsites] +
                     lattice[(i - N) % nsites] + lattice[(i - 1) % nsites])
        pair_sum += s * neighbour
        field_sum += s
    return -0.5 * eps_eff * pair_sum - B_eff * field_sum

def metropolis(N, X, T, eps=1.0, B=0.0):
    lattice = lattice_f(N)
    net_spin = np.zeros(X)
    net_energy = np.zeros(X)

    spin = np.sum(lattice)
    energy = energy_f(lattice, N, eps, B)
    net_spin[0] = spin
    net_energy[0] = energy

    nsites = N**2
    for i in range(1, X):
        x = random.randint(0, nsites-1)
        spin_i = lattice[x]
        neighbour = (lattice[(x + N) % nsites] + lattice[(x + 1) % nsites] +
                     lattice[(x - N) % nsites] + lattice[(x - 1) % nsites])
        # scale eps and B internally for ±1/2 spins so user-supplied eps,B match ±1 model
        eps_eff = 4.0 * eps
        B_eff = 2.0 * B
        dE = 2.0 * eps_eff * spin_i * neighbour + 2.0 * B_eff * spin_i
        r = random.random()
        if dE <= 0.0 or r <= math.exp(-dE / T):
            lattice[x] = -lattice[x]
            spin -= 2.0 * spin_i
            energy += dE
        net_spin[i] = spin
        net_energy[i] = energy
    return (net_spin) / (N**2), net_energy / (N**2)

In [3]:
# initial conditions (±1/2 spins)
N = 50
X = 2000*N**2
T = 2.2
eps = 1.0
B = 0

# exact Onsager (±1) and scaled to ±1/2 for plotting
# Only works for B = 0
def m_onsager(J, T):
    Tc = 2*J / math.log(1 + math.sqrt(2))
    if T >= Tc:
        return 0.0
    x = math.sinh(2 * J / T)
    return (1.0 - x**(-4))**(1.0/8.0)
meanm_pm1 = m_onsager(eps, T)
meanm = 0.5 * meanm_pm1  # scale to ±1/2 convention

# calculate magnetization and energy
magnetization, energy = metropolis(N, X, T, eps, B)
print(len(magnetization))

sweep = np.linspace(0, X/N**2, len(magnetization))  # use Monte Carlo sweeps
print(len(sweep))

print('Theoretical value of magnetization (±1/2):', meanm)

# plot magnetization as function of sweeps
plt.figure()
plt.axhline(y = meanm, color = 'r', linestyle = '-')
plt.axhline(y = -meanm, color = 'r', linestyle = '-')

plt.plot(sweep, magnetization)
plt.title('Ising model: Magnetization (±1/2)', fontsize=16)
plt.xlabel('Sweep', fontsize=16)
plt.ylabel('Magnetization per spin', fontsize=16)
plt.legend(labels = ['Theoretical value'])

# Calculate the mean of the numerical data for magnetization, starting from tau_eq (in attempted flips)
tau_eq = 500 * N**2
mean_m = np.mean(magnetization[tau_eq:])
print('The mean of the magnetization is:', mean_m)
print('The exact value of the magnetization is:', meanm)

magnetization_every_N2 = []
sweep_every_N2 = []
for i in range(0, len(magnetization), N**2):
    magnetization_every_N2.append(magnetization[i])
    sweep_every_N2.append(sweep[i])

print(len(sweep_every_N2))
print(len(magnetization_every_N2))

KeyboardInterrupt: 